# Goos-Hanchen experiment
***

DO NOT CHANGE, KEEP FOR LEGACY PURPOSES!

This notebook contains the data reduced from the usual Larmor Mantid reduction file. The experiment took place at Larmor beamline at ISIS from Nov 14-20th 2023. The reduced data is also contained in this github repository along with copies of the experiment log and pictures of the physical setup.

While the goal is to measure the Goos-Hanchen spatial shift of the two neutron spin states, it seems at the moment that we can only experimentally measure the phase shift between the two spin states. Again, the issue of the intrinsic coherence length of neutron pops up: Quan has shown some interesting potentially-observable effects in the different regiemes, but it is unclear how to exactly experimentally produce his proposed setups.

After performing the experiment, we found a noticable enhancement of absorption for one spin state, consistent with the idea that the multilayer is acting like a Fabry-Perot interferometer.

***
### Sample information
The two identical samples (assuming natural abundances) consist of the following layers with scattering length densities (SLDs) given in units of 10^-6 A^-2:
1. 50 nm Ti, SLD = -1.910
2. 50 nm Fe-Co-V 50-48-2%:
   1. Nuclear: SLD = 5.121
   2. Magnetic: SLD = -5.451 and 5.409
   3. Total: SLD = -0.33 and 10.53
4. 50 nm Si, SLD = 2.073
5. 50 nm Fe-Co-V 50-48-2%
6. 100 nm Ni-Mo 88-12% (non-magnetic), SLD = 8.52
7. Si substrate (total size 100 mm x 50 mm x 0.78 mm)

The sample can be magnetized in a ~200 G field.

We also have two identical non-magnetic blanks:
1. 100 nm Ni-Mo 88-12%
2. Si substrate (total size 100 mm x 50 mm x 0.78 mm)

***
### Problems/questions:
1. The film's easy axis and the plane of incidence are orthogonal: therefore, the beam size on target will be cut in half if we want the remnant mangetization to be parallel to the guide field at the sample position (vertical direction)
   1. ~~We can stack the two films back-to-back to create a better return loop for the magnetic flux lines at the sample position.~~
   2. ~~We can also try wrapping the sample in a thin layer of iron foil to close the flux loop.~~
   3. After discussion, we think that this will not be a problem because the sample is so thin, the dipole field should fall to zero quickly enough to be negligible.
2. Do we need a slit after the sample?
   1. Yes, to remove the direct beam

## Functions and constants

In [ ]:
import numpy as np
import sympy as sym
import scipy as sc
from scipy.interpolate import CubicSpline,make_smoothing_spline
from scipy.optimize import curve_fit
import sympy as sym
import matplotlib.pyplot as plt
from matplotlib import cm
import scienceplots
import csv
import warnings
from os.path import isdir,isfile

plot_width = 10
plot_height = plot_width/2
plt.style.use(['science','notebook', 'grid','default','bright'])
SMALL_SIZE,MEDIUM_SIZE,BIGGER_SIZE = 15,18,20
plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
plt.rc('axes', titlesize=BIGGER_SIZE)    # actually gets title font size

default_plot_parms = {'size':(plot_width,plot_height),'title':None,'xlabel':None,'ylabel':None,\
                      'grid':True,'figs':(1,1)}
def prepare_plot(parms=default_plot_parms):
    '''Helper function that returns basic figure object.'''
    fig,ax = plt.subplots(*parms['figs'],figsize=parms['size'])
    plt.title(parms['title'])
    plt.xlabel(parms['xlabel'])
    plt.ylabel(parms['ylabel'])
    plt.grid(parms['grid'])
    return fig,ax

### Legacy plotting methods ###
DEFAULT_PLOT_PARMS = {'title':'','xlab':'','ylab':'','size':(8,5),'lw':3,'ls':'--'}
SLD_PLOT_PARMS = DEFAULT_PLOT_PARMS|{'xlab':'Depth [AA]','ylab':'SLD [1e-6 AA^-2]','labels':['']}
THETA_C_PARMS = DEFAULT_PLOT_PARMS|{'xlab':r'$\lambda$ [AA]','ylab':r'$\theta_c$ [deg]',\
                                    'labels':['nuc','up','down','blank'],'ls':'-'}
DEFAULT_2DPLOT_PARMS = DEFAULT_PLOT_PARMS|{'title':'Pol','size':(12,8),'vals':(None,None),'extent':None,'xlab':r'$\lambda$',\
                                          'ylab':'Pixel','norm_max':False,'aspect':None,'shrink':.25}

DEFAULT_POL_PLOT_PARMS = {'title':'Pol','xlab':r'$\lambda$','ylab':r'Pixel','size':(12,8),'vals':(-1,1),\
                  'norm_max':False,'aspect':.25,'shrink':.25}

def plot_function(xdata,ydata,plot_parms=DEFAULT_PLOT_PARMS):
    '''General function to plot given data.'''
    assert len(xdata) == len(ydata), 'Len mismatch between x and y data!'
    fig,ax = plt.subplots(figsize=plot_parms['size'])
    
    for indx,(x,y) in enumerate(zip(xdata,ydata)):
        ax.plot(x,y,plot_parms['ls'],linewidth=plot_parms['lw'],label=plot_parms['labels'][indx])
    ax.set(title=plot_parms['title'],xlabel=plot_parms['xlab'],ylabel=plot_parms['ylab'])
    if plot_parms['labels'] is not None: plt.legend()

    plt.grid()
    plt.tight_layout()
    plt.show()

def plot_2D(image,plot_parms=DEFAULT_2DPLOT_PARMS):
    '''General function to plot 2D data.'''
    fig,ax = plt.subplots(figsize=plot_parms['size'])
    if plot_parms['norm_max']: vm = np.max(image)
    else: vm = plot_parms['vals'][-1]
    plt.imshow(image,origin='lower',vmin=plot_parms['vals'][0],vmax=vm,\
               extent=plot_parms['extent'],aspect=plot_parms['aspect'])
    ax.set(title=plot_parms['title'],xlabel=plot_parms['xlab'],ylabel=plot_parms['ylab'])
    plt.colorbar(shrink=plot_parms['shrink'])
    plt.show()

## Analytic calculations

In [ ]:
def get_su2_state(theta=0,phi=0,alpha=None,beta=None,gamma=0):
    '''Returns spin state, either with the usual parameterization or two complex numbers.'''
    if alpha is None or beta is None:
        result = [sym.cos(theta/2),sym.sin(theta/2)*sym.exp(sym.I*phi)]
    else:
        norm = sym.sqrt(sym.abs(alpha)**2 + sym.abs(beta)**2)
        result = 1/norm*[alpha,beta]
    return sym.exp(sym.I*gamma)*sym.Matrix(result).applyfunc(sym.simplify)

def get_op(a=0,b=0,c=0,d=0):
    '''Returns general operator in terms of the pauli vector.'''
    result = a*sym.eye(2) + b*sym.Matrix([[0,1],[1,0]]) + \
    c*sym.Matrix([[0,-sym.I],[sym.I,0]]) + d*sym.Matrix([[1,0],[0,-1]])
    return result.applyfunc(sym.simplify)

def get_proj(op,sign=1):
    '''Returns up/down projection operator.'''
    assert sign == -1 or sign == 1, 'The sign must be +/-1.'
    return (sym.Rational(1,2)*(sym.eye(2) + sign*op)).applyfunc(sym.simplify)

def expect(lstate,op,rstate=None,force_real=True):
    '''Returns the expectation value from the given state and operator.'''
    if rstate is None: rstate = lstate
    result = (lstate.H*op*rstate)[0]
    if force_real: result = sym.re(result)
    return result.simplify()

def change_basis(op,new):
    '''Returns operator in new basis.'''
    return (new.inv()*op*new).applyfunc(sym.simplify)

def time_av(func,var,N=1):
    '''Returns time average over N periods'''
    integral = func.integrate((var,-sym.pi*N,sym.pi*N))
    return (integral/(2*sym.pi*N)).expand().simplify()

p0,p1,p2,t1,t2,P1,P2,P = sym.symbols(r'\phi_0,\phi_1,\phi_2,\theta_1,\theta_2,\Phi_1,\Phi_2,\Phi',real=True)
idt,spx,spy,spz = get_op(a=1),get_op(b=1),get_op(c=1),get_op(d=1)

#asummes the magnetization is along the x-direction and incident polarization is along z
x_cob = 1/sym.sqrt(2)*sym.Matrix([[1,1],[1,-1]])
interaction = change_basis(sym.diag(1,sym.exp(sym.I*P)),x_cob)
print('Interaction matrix:')
display(interaction.applyfunc(sym.simplify))

In [ ]:
print('Evolution of the spin states:')
psi0 = get_su2_state(theta=sym.pi/2,phi=0)  #phi_0=0 for incident polarization along x-direction
display('Initial state:',psi0)
psi1 = get_op(a=(1+sym.exp(sym.I*t1))/2,d=(1-sym.exp(sym.I*t1))/2)*psi0
display('First arm:',psi1)
psi2 = interaction*psi1
display('After interaction:',psi2)
psi3 = get_op(a=(1+sym.exp(-sym.I*t2))/2,d=(1-sym.exp(-sym.I*t2))/2)*psi2
display('Second arm:',psi3)

In [ ]:
Pols = []
print('Final polarization components')
for i,k in zip(['Px','Py','Pz'],[spx,spy,spz]):
    print(f'{i}:')
    pk = expect(psi3,k).subs(p0,0).expand().simplify()
    display(pk)
    Pols.append(pk)  #saving this value
    print('At echo, theta_1 = theta_2')
    pk = pk.subs(t2,t1).expand().simplify()
    display(pk)
    print('Echo averaged')
    pk = time_av(pk,t1)
    display(pk)
    print('\n')

In [ ]:
Pols2 = []
print('Final polarization components')
for i,k in zip(['P_+x','P_-x','P_x'],[get_proj(spx,sign=1),get_proj(spx,sign=-1),spx]):
    print(f'{i}:')
    pk = expect(psi3,k).subs(p0,0).expand().simplify()
    display(pk)
    Pols2.append(pk)  #saving this value
    print('When theta_1 = -theta_2')
    pk = pk.subs(t2,-t1).expand().simplify()
    display(pk)
    print('Echo averaged')
    pk = time_av(pk,t1)
    display(pk)
    print('\n')

In [ ]:
nm,theta,phi,gam = sym.symbols(r'n_m,\theta,\phi,\gamma',real=True)  #general interaction matrix calculation
c1,c2 = get_su2_state(theta=theta,phi=phi,gamma=gam),\
        get_su2_state(theta=sym.pi-theta,phi=phi+sym.pi,gamma=-gam)
gen_basis = sym.Matrix(np.hstack([c1,c2]))
display(gen_basis)
display(gen_basis := gen_basis.subs(((theta,sym.pi/2),(phi,sym.pi/2),(gam,sym.pi/4))))

gen_mag = change_basis(spz,gen_basis).applyfunc(sym.simplify)
display(gen_mag)

#display(gen_mag)

## Numerical reflectivity calculations

This section needs some work, right now it doesn't agree with what Victor and Roger calculated.

In [ ]:
LAMS = np.linspace(2.5,13.5,300)  #nominal wavelength range on Larmor

def get_crit(SLD,lam=LAMS,degrees=True):
    '''Returns critical angle.'''
    result = np.arcsin(lam*np.sqrt(SLD*10**-6/np.pi))
    if degrees: result *= 180/np.pi
    return result

def get_pot(SLDs=[],ts=[],plot=False,plot_parms=SLD_PLOT_PARMS):
    '''Returns the potential given the thicknesses and SLDs of each layer.
    Doesn't include substrate.'''
    total_x = np.linspace(0,np.sum(ts),500)
    steps = np.array([np.sum(ts[:i]) for i in range(len(ts)+1)])
    potential = np.sum([SLDs[indx]*((steps[indx]<=total_x)&(total_x<=steps[indx+1])) for indx in range(len(SLDs))],axis=0)
    if plot: plot_function([total_x],[potential],plot_parms)
    return potential,total_x

SLDs = {'Ti':-1.910,'FeCoV_nuc':5.121,'FeCoV_up':-0.33,'FeCoV_down':10.53,'Si':2.06,'NiMo':8.52,\
       'exp_FeCoV_up':0.26961,'exp_FeCoV_down':9.93501}
sample_ts = np.array([50.,50.,50.,50.,100.])*10  #convert to Angstroms
blank_ts = np.array([100.])*10

sample_nuc_SLDs = [SLDs['Ti'],SLDs['FeCoV_nuc'],SLDs['Si'],SLDs['FeCoV_nuc'],SLDs['NiMo']]
sample_up_SLDs = [SLDs['Ti'],SLDs['FeCoV_up'],SLDs['Si'],SLDs['FeCoV_up'],SLDs['NiMo']]
sample_down_SLDs = [SLDs['Ti'],SLDs['FeCoV_down'],SLDs['Si'],SLDs['FeCoV_down'],SLDs['NiMo']]
blank_SLDs = [SLDs['NiMo']]

sample_tc = get_crit(sample_nuc_SLDs[1]),get_crit(sample_up_SLDs[2]),get_crit(sample_down_SLDs[1])
blank_tc = get_crit(blank_SLDs[0])
plot_function([LAMS for i in range(4)],[*sample_tc,blank_tc],plot_parms=THETA_C_PARMS)

sample_nuc,X = get_pot(sample_nuc_SLDs,ts=sample_ts,\
                       plot_parms=SLD_PLOT_PARMS|{'title':'Sample','labels':('nuc','up','down')})
sample_up,_ = get_pot(sample_up_SLDs,ts=sample_ts,\
                      plot_parms=SLD_PLOT_PARMS|{'title':'Sample','labels':('up')})
sample_down,_ = get_pot(sample_down_SLDs,ts=sample_ts,\
                        plot_parms=SLD_PLOT_PARMS|{'title':'Sample','labels':('down')})
blank,Xb = get_pot(blank_SLDs,ts=blank_ts,\
                   plot_parms=SLD_PLOT_PARMS|{'title':'Sample','labels':('blank')})

plot_function([X,X,X,Xb],[sample_nuc,sample_up,sample_down,blank],\
              plot_parms=SLD_PLOT_PARMS|{'title':'Sample','labels':('nuc','up','down','blank')})

In [ ]:
sample_up_SLDs = [SLDs['Ti'],SLDs['exp_FeCoV_up'],SLDs['Si'],SLDs['exp_FeCoV_up'],SLDs['NiMo']]
sample_down_SLDs = [SLDs['Ti'],SLDs['exp_FeCoV_down'],SLDs['Si'],SLDs['exp_FeCoV_down'],SLDs['NiMo']]

sample_up,_ = get_pot(sample_up_SLDs,ts=sample_ts,\
                      plot_parms=SLD_PLOT_PARMS|{'title':'Sample','labels':('up')})
sample_down,_ = get_pot(sample_down_SLDs,ts=sample_ts,\
                        plot_parms=SLD_PLOT_PARMS|{'title':'Sample','labels':('down')})
plot_function([X,X],[sample_up,sample_down],\
              plot_parms=SLD_PLOT_PARMS|{'title':'Sample','labels':('up','down')})

In [ ]:
def get_t_matrix(SLDs,ts,k0=.314,substrate=SLDs['Si']):
    '''Given a potential, returns the transfer matrix. Assumes air for z<=0 and 
    an additional infinite-thickness substrate layer.'''
    assert len(SLDs) == len(ts), 'Check number of layers!'
    km = [np.emath.sqrt(k0**2 - 4*np.pi*rho*1e-6) for rho in [0.]+SLDs+[substrate]]
    ms = len(km)  #air 0, layers (1,...,N), substrate N+1
    
    phim = [0]+[km[i]*ts[i] for i in range(ms-2)]  #phi_0 special case
    
    tm = [2*km[i]/(km[i] + km[i+1]) for i in range(ms-1)]
    rm = [(km[i] - km[i+1])/(km[i] + km[i+1]) for i in range(ms-1)]

    Rm = [np.array([[1,rm[i]],[rm[i],1]]) for i in range(ms-1)]
    Cm = [1/tm[i]*np.diag((np.exp(-1j*phim[i]),np.exp(1j*phim[i])))@Rm[i] for i in range(ms-1)]

    t_matrix = np.identity(2)
    for C in Cm:
        t_matrix = np.matmul(t_matrix,C)
    return t_matrix

def get_R_Phi(t_up,t_down=None):
    '''Calculates reflectivity (R) and phase (Phi) from t-matrix entries.'''
    R_up = np.abs(t_up[:,1,0])**2/np.abs(t_up[:,0,0])**2
    result = [R_up]
    if t_down is not None:
        R_down = np.abs(t_down[:,1,0])**2/np.abs(t_down[:,0,0])**2
        result.append(R_down)
        result.append(np.angle(t_up[:,1,0]/t_up[:,0,0]) - np.angle(t_down[:,1,0]/t_down[:,0,0]))
    return result

theta = .35*np.pi/180  #scattering angle
wavelengths = np.linspace(2,13.5,1000)

t_up = np.array([get_t_matrix(sample_up_SLDs,sample_ts,k0=2*np.pi*theta/w) for w in wavelengths])
t_down = np.array([get_t_matrix(sample_down_SLDs,sample_ts,k0=2*np.pi*theta/w) for w in wavelengths])
R_up,R_down,phase = get_R_Phi(t_up,t_down)

plt.figure(figsize=(10,5))
plt.plot(wavelengths,np.cos(phase),'-',label='Pol',lw=2,color='C0')
plt.plot(wavelengths,R_up,'--',label='$R_{up}$',lw=2,color='C1')
plt.plot(wavelengths,R_down,'--',label='$R_{down}$',lw=2,color='C2')
#plt.plot(wavelengths,R_nuc,'--',label='$R_{nuc}$',lw=2,color='C1')
#plt.plot(wavelengths,R_blank,'--',label='$R_{blank}$',lw=2,color='C2')
plt.xlabel(r'$\lambda$ (AA)')
plt.ylim(-1.05,1.05)
plt.legend()
plt.grid()
plt.show()

'''GH_shift = wavelengths[1:]**2*np.cos(theta*np.pi/180)*np.diff(phase)/(2*np.pi)
plt.figure(figsize=(10,5))
plt.plot(wavelengths,phase,'--',label=r'$\Phi$',lw=2,color='C0')
plt.plot(wavelengths[1:],GH_shift,'-',label='GH-shift',lw=2,color='C1')
plt.xlabel(r'$\lambda$ (AA)')
plt.legend()
plt.grid()
plt.show()'''

## Experiment log
***

### 11/14 Day One (Tuesday)
Setup and calibration of the beamline.

* 9:30
* We decided that the magnetization of the sample will probably not be a problem because the sample is so thin. Therefore we will not worry about the mismatch of the sample field direction and the guide field direction.
* Still need to magnetize the sample today and finish the sample holder.

* 2:00
* Found the echo before lunch, need to finish building the sample holder and check the echo as we scan the sample stage (should only need a few mm of drive)
* Should we use a supermirror or a blank for tuning echo? (decided to just use blank)
* Aligning the sample slit with the transmission monitor

* 7:00
* Decided to run echo scans overnight to test the long-term stability of the echo point because we cannot retune the echo once we put the magnetic sample in the beam. Also included a scan of the sample stack translation.

### 11/15 Day Two (Wednesday)
Mounting the blank with NO longitudinal splitting. (and the sample)

* 10:30
* Echo stability was very stable overnight for all positions of the sample stack. Above 80% for 3-10 AA.
* Leveled the sample stack (+/- 0.1 degrees for both directions)
* Adjusted the sample center of rotation to be the same center of rotation of the second arm (within few cms).
* Finding the center of the beam using a piece of Cd taped to the Al sample holder
* By eye using laser, the center is at 27 mm and 58.25 degrees
* Laser beam and neutron beam are off by 0.4 mm using the monitor right after the sample position (center 26.6 mm using neutrons, add 0.8 mm to include the thickness of the sample)
* Increasing the beam height did not change the tune (25 mm at the front, 40 mm at the sample)
* Collimation from pipe 31 mm
* Snout aperture 30 mm height 30 mm width (yes, it is that wide)
  

* 11:00
* Pixel 34 is the center of the beam at the moment (0.64 mm per pixel)
* 41004,41005 are the spectra numbers
  

* 1:00
* Magnetized the sample, wrote the field direction on the Al plate. Went to above 0.027 Tesla
* Found the critical edge to be at 0.35 degrees (aiming for 0.4 degrees?)

* 3:00
* The blank appears to be curved? The beam is spread over 8 pixels instead of 4 as before.

* 5:00
* Aperture width is still 2 mm wide
* Magnetic sample agrees well with Victor's model assuming 89% magnetization. This value agrees more closely with the 
* Going to measure the magnetic sample overnight

### 11/16 Day Three (Thursday)

* 10:00
* Magnetic sample ran successfully overnight. Improving statistics on the blank until after lunch.

* 8:00
* Magnetic sample rotated by 90 degrees replicated the original polarization except now the signal is cos(phi) instead of 1/2*(1 + cos(phi))
* Going to run this orientation overnight and add the grating in tomorrow

### 11/17 Day Four (Friday)

* 11:00
* Added the grating after the sample (GR27, 2 um period, 10.5 um depth, 560 nm groove width)
* Sample still in the vertical direction
* Doubled-checked where the instrument parameters are stored. They appear in the log book under "Analysis/Property/Run".
  * Pre-sample slit: 0.5 mm by 25 mm
  * Post sample slit: 2 mm by 25 mm

### 11/18 Day Five (Saturday)

* 10:00
* Running with grating before sample until 2:00 pm to compare with the sample only run
* After that, we'll get better statistics for the P0 run before changing the incident angle to shift the GH bump in the polarization to appear at shorter wavelengths
* The journal viewer title was wrong for some runs. The correct naming is
  * 79130 - 79142 No grating
  * 79143 - 79144 Grating after sample
  * 79145 - 79159 Grating before sample
  * 79160 - 79164 Grating after sample (again)

* 11:30
* Going to measure the broken samples with X-rays (don't know the roughness numbers)
* Got the fitting functon for P0 to work, need to port the q function as well
* The new data seems show that the grating was slightly tilted about the vertical axis or that the P0 has slightly decreased (more likely grating angle misalignment because peak heights are still tall)

### 11/19 Day Six (Sunday)

* 9:00
* Overnight data looks good. The peak shifted from ~8.5 A to 11 A as expected
* Running with grating before the sample, will run until 8 pm tonight, then put the grating after the sample to finish through the night.
* Results at 0.4 consistent with the old 0.35 degree data
* 0.40 degree data:
  * 79179 - 79188 No grating
  * 79189 - 79199 Grating before sample
  * 79200 - 79212 Grating after sample

## Post-experiment data analysis
***

### Data loading and processing functions

In [ ]:
data_paths = {'laptop':'C:\\Users\\samck\\Documents\\GitHub\\Goos-Hanchen\\\Reduced_data_correct_eb\\',
             'ceem':'C:\\Users\\samckay\\Documents\\GitHub\\Goos-Hanchen\\\Reduced_data_correct_eb\\',
             'home':'C:\\Users\\samck\\Documents\\GitHub\\Goos-Hanchen\\\Reduced_data_correct_eb\\'}

DEFAULT_PATH = data_paths['ceem']
assert isdir(DEFAULT_PATH), 'Default data directory was not found!'

def load_data(file,path=DEFAULT_PATH):
    '''Loads the mantid output data into numpy array.'''
    assert isdir(path), 'Data directory was not found!'
    assert isfile(path+file), 'Data file was not found!'
    with open(path+file) as f:
        temp = f.readlines()[2:-2]
        temp = [l.replace('\n','').split(',') for l in temp]
        x = np.array([float(temp[x][0]) for x in range(len(temp))])
        y = np.array([float(temp[y][1]) for y in range(len(temp))])
        e = np.array([float(temp[y][2]) for y in range(len(temp))])
    return x,y,e

def load_data_2D(file,path=DEFAULT_PATH):
    '''Loads the mantid output data into numpy 2D array.
    Returns X, Y, and E coordinate arrays, and pixel name.'''
    assert isdir(path), 'Data directory was not found!'
    assert isfile(path+file), 'Data file was not found!'
    with open(path+file) as f:
        lines = f.readlines()[1:]
        lines = [l.replace('\n','').split(',') for l in lines]
        pnames,xs,ys,es = [],[],[],[]  #pixel name from DAC
        for l in lines:
            if len(l) != 3: pnames.append(l[0])
            else:
                xs.append(float(l[0]))
                try:
                    ys.append(float(l[1]))
                    es.append(float(l[2]))
                except Exception:
                    ys.append(np.NaN)  #dumbass Mantid stuff
                    es.append(np.Inf)
        lnum,pnum = len(lines),len(pnames)  #tof pixels, spatial pixels
        new_shape = (len(pnames),(len(lines)-len(pnames))//len(pnames))
    return np.array(xs).reshape(new_shape),np.array(ys).reshape(new_shape),np.array(es).reshape(new_shape),pnames

def trim_arr(arrays,segments):
    '''Applied the same trim to a list of arrays. First array determines
    the trimming for the other arrays.'''
    result = []
    masks = np.array([(s[0]<arrays[0]) & (arrays[0]<s[1]) for s in segments])

    if masks[0].ndim == 1: new_pix_nums = int(np.sum(masks[0]))
    else: new_pix_nums = masks[0].shape[0],np.sum(masks[0],axis=1)[0] #number of new vertical and horizontal pixels
    for a in arrays:
        for m in masks:
            a = a[m]
        result.append(a)
    return [r.reshape(new_pix_nums) for r in result]

def show_image(image,x,y,title=None,x_label=None,y_label=None,vmin=None,vmax=None,\
              size=(13,6),shrink=.25,aspect=.4,phi=-60,theta=30):
    """Plots the given 2D image with a heat map and 3D plot."""
    fig = plt.figure(figsize=size)
    ax1 = fig.add_subplot(1,2,1)
    im1 = ax1.imshow(image,origin='lower',vmin=vmin,vmax=vmax,aspect=aspect,extent=(x[0],x[-1],y[0],y[-1]))
    plt.colorbar(im1,ax=ax1,shrink=shrink)
    ax1.set_xlabel(x_label)
    ax1.set_ylabel(y_label)

    ax2 = fig.add_subplot(1,2,2,projection='3d')
    ax2.view_init(elev=theta, azim=phi, roll=0)
    X,Y = np.meshgrid(x,y)
    im2 = ax2.plot_surface(X,Y,image,cmap=cm.viridis,vmin=vmin,vmax=vmax)
    ax2.set_xlabel(x_label,labelpad=7.)
    ax2.set_ylabel(y_label,labelpad=7.)
    
    plt.suptitle(title)
    plt.tight_layout()
    plt.show()

def get_center(up,down,pname,rng=7,show=True):
    '''Returns the center pixel of the given data (assumed to be from blank).'''
    assert rng%2 == 1,'Pixel range (rng) must be odd.'
    sum1,sum2 = np.nansum(up,axis=1),np.nansum(down,axis=1)
    W = np.arange(len(pname))  #pixel number are the weights
    c1,c2 = np.average(W,weights=sum1),np.average(W,weights=sum2)
    center = int(round(np.mean([c1,c2])))
    pixels = [center-(rng-1)//2,center+(rng+1)//2]

    if show:
        print('Center pixel:')
        print(f'[up,down]: {np.round([c1,c2],5)} -> {center}')
        print('Pixels used:')
        print([p for p in range(*pixels)])
        plt.figure(figsize=(7,3))
        plt.ylabel('shim')
        plt.xlabel('pixel')
        plt.plot(sum1)
        plt.plot(sum2)
        plt.axvline(x=center,ls='--',color='blue')
        plt.axvline(x=pixels[0],ls='--',color='red')
        plt.axvline(x=pixels[-1]-1,ls='--',color='red')
        plt.grid()
        plt.show()
        
    return center,pixels  #assumed that rightmost point is NOT included

def get_pol_err(Pol,Shim,ff=1e4):
    '''Direct method of getting polarization error.'''
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")  #ignores nan mulitply and zero divide warnings
        err = np.sqrt((1 - Pol**2)/(2*Shim*ff))
    return np.where(err<.5,err,np.Inf)

def do_sig_fit(data,wavelengths,tests=[116,125],use_shim=True):
    '''Performs a fit of the standard deviations as a function of wavelength.'''
    def gauss(x,amp,av,sig,shim):
        '''Simple gaussian function.'''
        return amp*np.exp(-((x - av)/sig)**2/2) + shim

    pixels,_ = np.shape(data)
    sigs,errs = [],[]
    plt.figure(figsize=(8,4))
    for indx,lam in enumerate(data.T):
        f = gauss
        guess = [np.max(lam),np.argmax(lam),np.sqrt(np.cov(range(pixels),aweights=lam)),np.min(lam)/2]
        bnds = [[0.,1.,0.,0.],[2*np.max(lam),pixels,pixels,2*np.max(lam)]]
        if not use_shim:
            f = lambda x,amp,av,sig: gauss(x,amp,av,sig,0)
            guess = guess[:-1]
            bnds[0],bnds[-1] = bnds[0][-1],bnds[-1][-1]
        try: 
            parms,cov = curve_fit(f,range(pixels),lam,p0=guess,bounds=bnds)
        except Exception:
            print(f'Number {indx} failed!')
            parms = [0]*len(guess)
            cov = np.zeros((len(guess),len(guess)))
        sigs.append(parms[2])
        errs.append(np.sqrt(np.diag(cov))[2])
        if tests[0] <= indx < tests[-1]:
            plt.plot(lam,'o',color=f'C{indx}',label=f'{round(wavelengths[indx],3)}')
            x = np.linspace(0,pixels-1,200)
            plt.plot(x,f(x,*parms),'-',color=f'C{indx}')
    ax = plt.gca()
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    plt.grid()
    plt.show()

    return np.array(sigs),np.array(errs),tests

def get_pol(up,down,trim_error=None,rev=True):
    '''Calculates polarization.'''
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")  #ignores nan mulitply and zero divide warnings
        Pol_blank = (up - down)/(up + down)
        if rev: Pol_blank = -1*Pol_blank
        Err_blank = 2/(up + down)**2*np.sqrt(up**2*down + down**2*up)
        if trim_error is not None:
            Err_blank = np.where(Err_blank>trim_error,trim_error,Err_blank)
    return Pol_blank,Err_blank

def get_shim(up,down,show=True,plot_parms=DEFAULT_POL_PLOT_PARMS|{'title':'Shim','vals':(None,None),'log vals':(None,None)}):
    '''Returns the shim intensity.'''
    assert np.shape(up) == np.shape(down),'Up and down counts must have equal shapes!'
    shim = up + down
    pixels = np.arange(len(shim[:,0]))
    std_shim = np.array([np.sqrt(np.cov(pixels,aweights=lam)) for lam in shim.T])
    if show:
        plot_2D(shim,plot_parms=plot_parms)
        new_title = plot_parms['title'] + ' (log)'
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")  #ignores nan mulitply and zero divide warnings
            plot_2D(np.log(shim),plot_parms=plot_parms|{'title':new_title,'vals':plot_parms['log vals']})
    return shim,std_shim

def Q_conversion(lam,pixels,center,th0,pix_size=.65,dist=4350.):
    '''Returns conversion matrix of wavelength to momentum transfer.'''
    thQ = np.array([.5*np.arctan((p - center)/dist*pix_size) for p in range(pixels)])
    Q = np.array([4*np.pi/lam*np.sin(th0*np.pi/180. - thq) for thq in thQ])
    return np.flip(Q,axis=1)

def do_rebin(data,new_bins):
    '''Rebins interpolated data for constant q so each pixel has the same binning.'''
    temp = np.full(len(data),np.NaN)
    for old,new in enumerate(new_bins):
        temp[new] = data[old]
    return temp

def get_constant_Q(data,errs,lam,th0,trim,q_bins=400,show_all=True,show_all_intrp=False,\
                   show_final_intrp=True,show_sum=True,lims=(-1.1,1.1)):
    '''Returns and plots the input data rebinned with constant q.
    Uses a simple cubic spline for the interpolation.
    Returns the linearly-binned wavelength and constant q data.'''
    pixels,tpixs = len(data[:,0]),len(data[trim[0]:trim[-1],0])
    Q = Q_conversion(lam,pixels,(pixels-1)//2,th0)[trim[0]:trim[-1]]
    data_Q = np.flip(data,axis=1)[trim[0]:trim[-1]]  #need to flip for spline
    errs_Q = np.flip(errs,axis=1)[trim[0]:trim[-1]]
    q_spaces = [np.linspace(q[0],q[-1],q_bins) for q in Q]
    data_intrp = [CubicSpline(q,p)(qs) for q,p,qs in zip(Q,data_Q,q_spaces)]
    err_intrp = [CubicSpline(q,e)(qs) for q,e,qs in zip(Q,errs_Q,q_spaces)]

    q_tot = np.linspace(np.min(Q),np.max(Q),q_bins)
    new_bins = np.digitize(q_spaces,q_tot,right=True)
    const_q = np.array([do_rebin(p,nb) for p,nb in zip(data_intrp,new_bins)])
    const_q_errs = np.array([do_rebin(e,nb) for e,nb in zip(err_intrp,new_bins)])
    
    lam_const_q = 4*np.pi/q_tot*np.sin(th0*np.pi/180)  #convert to wavelength
    lam_bins = np.linspace(np.min(lam_const_q),np.max(lam_const_q),q_bins)
    second_intrp = np.array([CubicSpline(np.flip(lam_const_q),p)(lam_bins) for p in np.flip(np.nan_to_num(const_q),axis=1)])
    second_intrp_errs = np.array([CubicSpline(np.flip(lam_const_q),e)(lam_bins) for e in np.flip(np.nan_to_num(const_q_errs),axis=1)])

    temp_lam,second_intrp,second_intrp_errs = trim_arr((np.stack([lam_bins]*tpixs),second_intrp,second_intrp_errs),[(lam[0],lam[-1])])
    lam_bins = temp_lam[0]
    
    if show_all:
        plt.figure(figsize=(11,4))
        for indx,(q,p) in enumerate(zip(Q,data_Q)):
            plt.plot(q,p,'.-',label=f'Pix {indx}')
        plt.grid()
        plt.ylim(lims)
        plt.xlabel('q')
        plt.legend()
        plt.show()
    if show_all_intrp:
        for indx,(q,p) in enumerate(zip(Q,data_Q)):
            plt.figure(figsize=(11,4))
            plt.plot(q,p,'o',label=f'Pix {indx}')
            plt.plot(q_spaces[indx],CubicSpline(q,p)(q_spaces[indx]),'-',color='red')
            plt.grid()
            plt.ylim(lims)
            plt.xlabel('q')
            plt.legend()
            plt.show()
    if show_final_intrp:
        plt.figure(figsize=(11,4))
        for indx,p in enumerate(data_intrp):
            plt.plot(q_spaces[indx],p,label=f'Pix {indx}')
        plt.ylim(lims)
        plt.xlabel('q')
        plt.legend()
        plt.grid()
        plt.show()
    if show_sum:
        plt.figure(figsize=(11,4))
        plt.title('Constant-q binned')
        for indx,p in enumerate(const_q):
            plt.plot(lam_const_q,p,'.-',label=f'{indx}')
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")  #ignores nan mulitply and zero divide warnings
            plt.plot(lam_const_q,np.nansum(const_q,axis=0),'.-',color='black',label='Sum')
        plt.legend()
        plt.grid()
        plt.xlabel('Wavelength')
        plt.ylim(lims)
        plt.show()

    return lam_bins,np.where(second_intrp>0,second_intrp,0.),second_intrp_errs

def do_fit(Xt,Yt,Et=None,plot_each=False,plot_all=True,extended=(0.,13.),rebin=None):
    '''Performs and returns fit of P0 from the blank sample.
    This function uses the old variable naming conventions.'''
    chubby = np.polynomial.chebyshev.Chebyshev.fit
    get_chubby = np.polynomial.chebyshev.chebval
    chubby_x = np.linspace(extended[0],extended[-1],200)
    xt,yt = Xt[0,:],np.arange(len(Yt[:,0]))  #wavelength, pixel number

    new_xt = xt
    if rebin is not None:
        new_xt = rebin

    if Et is not None: 
        fitted = np.array([get_chubby(new_xt,chubby(xt,yt,2,w=1/et).convert().coef) for yt,et in zip(Yt,Et)])
        fitted_extended = np.array([get_chubby(chubby_x,chubby(xt,yt,2,w=1/et).convert().coef) for yt,et in zip(Yt,Et)])
    else:
        print('Errors were ignored!')
        Yt = np.nan_to_num(Yt)  #Why is the fit so bad without the errors?
        fitted = np.array([get_chubby(new_xt,chubby(xt,yt,2).convert().coef) for yt in Yt])  #no errors
        fitted_extended = np.array([get_chubby(chubby_x,chubby(xt,yt,2).convert().coef) for yt in Yt])
        Et = np.zeros(np.shape(Yt))
    
    if plot_each:
        for indx,(f,xt,yt,et) in enumerate(zip(fitted_extended,Xt,Yt,Et)):
            plt.figure(figsize=(11,4))
            plt.plot(xt,yt,color='C0')
            plt.errorbar(xt,yt,et,color='C0',ls='none',zorder=5)
            plt.plot(chubby_x,f,color='C1',zorder=10,label=f'Pixel {indx}')
            plt.legend()
            plt.grid()
            plt.show()
    if plot_all:
        plt.figure(figsize=(11,4))
        plt.title('$P_0$ fit')
        for indx,(f,xt,yt,et) in enumerate(zip(fitted_extended,Xt,Yt,Et)):
            #plt.plot(xt,yt,color='C0',label='Data')  #too messy...
            #plt.errorbar(xt,yt,et,color='C0',ls='none',zorder=5)
            plt.plot(chubby_x,f,zorder=10,label=f'Pix {indx}')
        ax = plt.gca()
        ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
        plt.grid()
        plt.show()
        
    return fitted,fitted_extended

### Fitting P0 from the blank sample data and choosing the beam-defining pixels

#### Manually calculating the polarization:

I first tried to use the up and down spin states directly, but the data variation seemed to large. I think there's an additional monitor and proton curent normalization somewhere in the Mantid code that I didn't take into account. To get the proper error bars, we need to understand this normalization.

In [ ]:
X,Yup,Eup,pname = load_data_2D(r"blank_perp\\up.txt")  #finding the center pixel
_,Ydn,Edn,_ = load_data_2D(r"blank_perp\\dn.txt")

plot_2D(Yup,plot_parms=DEFAULT_2DPLOT_PARMS|{'title':'Raw up counts','shrink':.5})
plot_2D(Ydn,plot_parms=DEFAULT_2DPLOT_PARMS|{'title':'Raw down counts','shrink':.5})

_,EDGES = get_center(Yup,Ydn,pname,rng=7)  #EDGES doesn't include right-most value

Xt,Yupt,Eupt = trim_arr((X,Yup,Eup),[(3.,12.)])  #trimming off the zero-count regions
Yupt = Yupt[EDGES[0]:EDGES[-1],:]

_,Ydnt,Ednt = trim_arr((X,Ydn,Edn),[(3.,12.)])
Ydnt = Ydnt[EDGES[0]:EDGES[-1],:]

LAMBDAS,PIXELS = Xt[0,:],np.arange(EDGES[-1]-EDGES[0])  #use this wavelength and pixel range for ALL data
SHIM0_PLOT_PARMS = {'title':'Shim','xlab':r'$\lambda$','ylab':r'Pixel','size':(12,8),'vals':(None,None),\
                  'aspect':.25,'extent':(LAMBDAS[0],LAMBDAS[-1],PIXELS[0],PIXELS[-1]),'norm_max':True,'shrink':.25,\
                   'log vals':(-8,-5)}

plot_2D(Yupt,plot_parms=DEFAULT_2DPLOT_PARMS|{'title':'Trimmed up','shrink':.25,'aspect':4})
plot_2D(Ydnt,plot_parms=DEFAULT_2DPLOT_PARMS|{'title':'Trimmed down','shrink':.25,'aspect':4})

Pol_blank,Err_blank = get_pol(Yupt,Ydnt)
show_image(Pol_blank,LAMBDAS,PIXELS,title='Pol blank (P0), trimmed',x_label='Wavelength',y_label='Pixel',vmin=.4,vmax=1)

In [ ]:
X,Yup,Eup,pname = load_data_2D(r"blank_para1\\up.txt")  #finding the center pixel
_,Ydn,Edn,_ = load_data_2D(r"blank_para1\\dn.txt")

plot_2D(Yup,plot_parms=DEFAULT_2DPLOT_PARMS|{'title':'Raw up counts','shrink':.5})
plot_2D(Ydn,plot_parms=DEFAULT_2DPLOT_PARMS|{'title':'Raw down counts','shrink':.5})

_,EDGES = get_center(Yup,Ydn,pname,rng=7)  #EDGES doesn't include right-most value

Xt,Yupt,Eupt = trim_arr((X,Yup,Eup),[(3.,12.)])  #trimming off the zero-count regions
Yupt = Yupt[EDGES[0]:EDGES[-1],:]

_,Ydnt,Ednt = trim_arr((X,Ydn,Edn),[(3.,12.)])
Ydnt = Ydnt[EDGES[0]:EDGES[-1],:]

LAMBDAS,PIXELS = Xt[0,:],np.arange(EDGES[-1]-EDGES[0])  #use this wavelength and pixel range for ALL data

Pol_blank,Err_blank = get_pol(Yupt,Ydnt)
show_image(Pol_blank,LAMBDAS,PIXELS,title='Pol blank (P0), trimmed',x_label='Wavelength',y_label='Pixel',vmin=.4,vmax=1)

In [ ]:
X,Yup,Eup,pname = load_data_2D(r"blank_para2\\up.txt")  #finding the center pixel
_,Ydn,Edn,_ = load_data_2D(r"blank_para2\\dn.txt")

plot_2D(Yup,plot_parms=DEFAULT_2DPLOT_PARMS|{'title':'Raw up counts','shrink':.5})
plot_2D(Ydn,plot_parms=DEFAULT_2DPLOT_PARMS|{'title':'Raw down counts','shrink':.5})

_,EDGES = get_center(Yup,Ydn,pname,rng=7)  #EDGES doesn't include right-most value

Xt,Yupt,Eupt = trim_arr((X,Yup,Eup),[(3.,12.)])  #trimming off the zero-count regions
Yupt = Yupt[EDGES[0]:EDGES[-1],:]

_,Ydnt,Ednt = trim_arr((X,Ydn,Edn),[(3.,12.)])
Ydnt = Ydnt[EDGES[0]:EDGES[-1],:]

LAMBDAS,PIXELS = Xt[0,:],np.arange(EDGES[-1]-EDGES[0])  #use this wavelength and pixel range for ALL data

Pol_blank,Err_blank = get_pol(Yupt,Ydnt)
show_image(Pol_blank,LAMBDAS,PIXELS,title='Pol blank (P0), trimmed',x_label='Wavelength',y_label='Pixel',vmin=.4,vmax=1)

In [ ]:
X,Yup,Eup,pname = load_data_2D(r"blank_all\\up.txt")  #finding the center pixel
_,Ydn,Edn,_ = load_data_2D(r"blank_all\\dn.txt")

plot_2D(Yup,plot_parms=DEFAULT_2DPLOT_PARMS|{'title':'Raw up counts','shrink':.5})
plot_2D(Ydn,plot_parms=DEFAULT_2DPLOT_PARMS|{'title':'Raw down counts','shrink':.5})

_,EDGES = get_center(Yup,Ydn,pname,rng=7)  #EDGES doesn't include right-most value

Xt,Yupt,Eupt = trim_arr((X,Yup,Eup),[(3.,12.)])  #trimming off the zero-count regions
Yupt = Yupt[EDGES[0]:EDGES[-1],:]

_,Ydnt,Ednt = trim_arr((X,Ydn,Edn),[(3.,12.)])
Ydnt = Ydnt[EDGES[0]:EDGES[-1],:]

LAMBDAS,PIXELS = Xt[0,:],np.arange(EDGES[-1]-EDGES[0])  #use this wavelength and pixel range for ALL data

Pol_blank,Err_blank = get_pol(Yupt,Ydnt)
show_image(Pol_blank,LAMBDAS,PIXELS,title='Pol blank (P0), trimmed',x_label='Wavelength',y_label='Pixel',vmin=.4,vmax=1)

The center pixel from the blank data is 34. For the rest of this analysis, we will use a total of 7 pixels; it looks like we may be able to extend by another pixel or two for some of the data, but then the long wavelengths really start to loose counts.

##### Another reason for being conservative on the beam-defining pixels is to ensure that the shim intensity for the blank is roughly constant, so can just used the mean of the P0 fit:

In [ ]:
Y_summed,_ = get_pol(np.sum(Yupt,axis=0),np.sum(Ydnt,axis=0))
plt.figure(figsize=(8,3))
plt.title('Comparing pre vs post pol binning for blank')
plt.plot(LAMBDAS,Y_summed,label='summed')
plt.plot(LAMBDAS,np.mean(Pol_blank,axis=0),'--',label='averaged')
plt.legend()
plt.grid()
plt.show()

#### The blank shim intensity and spread:

1.  Directly calculating the standard deviations at each wavelength is the easiest method
2.  Fitting each wavelength to a Gaussian is more involved, and I'm having issues with the fitting convergence because much of the data is too noisy

For the rest of the data, we will only consider the first method of direct calculation of the standard deviation (STD)

In [ ]:
SHIM0_PLOT_PARMS = {'title':'Shim','xlab':r'$\lambda$','ylab':r'Pixel','size':(12,8),'vals':(None,None),\
                  'aspect':.25,'extent':(LAMBDAS[0],LAMBDAS[-1],PIXELS[0],PIXELS[-1]),'norm_max':True,'shrink':.25,\
                   'log vals':(-8,-5)}

Shim_blank,STD_blank = get_shim(Yupt,Ydnt,plot_parms=SHIM0_PLOT_PARMS)  #first method
Shim_blank_summed = np.sum(Shim_blank,axis=0)

plt.figure(figsize=(10,4))
plt.title('Blank shim')
plt.xlabel('Wavelength')
for pix,s in enumerate(Shim_blank):
    plt.plot(LAMBDAS,s,label=f'Pix {pix}')
plt.plot(LAMBDAS,Shim_blank_summed,label='sum',color='black')
plt.grid()
ax = plt.gca()
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

plt.figure(figsize=(10,4))
plt.title('Blank shim standard deviation')
plt.xlabel('Wavelength')
plt.plot(LAMBDAS,STD_blank,'.-')
plt.grid()
plt.show()

blank_sigs,blank_errs,tests = do_sig_fit(Shim_blank,LAMBDAS,use_shim=True)  #second method

plt.figure(figsize=(10,4))
plt.title('Blank standard deviations')
plt.plot(LAMBDAS,blank_sigs,'.',color='C0',label='fitted')
plt.plot(LAMBDAS,STD_blank,'.-',color='C1',label='calc')
plt.errorbar(LAMBDAS,blank_sigs,yerr=blank_errs,color='C0',ls='')
plt.axvline(x=LAMBDAS[tests[0]],ls='--',color='black')
plt.axvline(x=LAMBDAS[tests[-1]],ls='--',color='black')
ax = plt.gca()
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.xlabel('Wavelength')
plt.ylim((0.,3.5))
plt.grid()
plt.show()

#### Fitting the blank polarization $(P_0)$ for each pixel:

Each pixel will be fit separately by a quadratic polynomial, specifically by the Chebyshev polynomials to minimize Runge's phenomenon.

In [ ]:
X,Y,E,pname = load_data_2D(r"blank\\pol.txt")

P0_PLOT_PARMS = {'title':'$P_0$ Pol','xlab':r'$\lambda$','ylab':r'Pixel','size':(12,8),'vals':(0,1),\
                  'extent':None,'norm_max':False,'aspect':None,'shrink':.25}

plot_2D(Y,plot_parms=P0_PLOT_PARMS)

Xt,Yt,Et = trim_arr((X,Y,E),[(3.,12.)])  #trimming wavelength

Yt = Yt[EDGES[0]:EDGES[-1],:]  #trimming pixels
Et = Et[EDGES[0]:EDGES[-1],:]
Xt = Xt[EDGES[0]:EDGES[-1],:]

plot_2D(Yt,plot_parms=P0_PLOT_PARMS|{'aspect':.25,'extent':(LAMBDAS[0],LAMBDAS[-1],PIXELS[0],PIXELS[-1]),'vals':(0,1)})

fitted,fitted_extended = do_fit(Xt,Yt,Et)  #used binning from Mantid

### Fitting $P_0$ for use with the constant-q binning:

Constant q binning assumes that the data is now truly given as a function of wavelength (after summing/averaging across pixels), so all we need to do is rebin the interpolated values from the polynomial fit.

In [ ]:
bl_lam,bl_up,bl_up_err = get_constant_Q(Yupt,Eupt,LAMBDAS,th0=.35,trim=(1,-1),lims=(None,None),\
                                        show_all=False,show_all_intrp=False,show_final_intrp=False,show_sum=False)
_,bl_dn,bl_dn_err = get_constant_Q(Ydnt,Ednt,LAMBDAS,th0=.35,trim=(1,-1),lims=(None,None),\
                                   show_all=False,show_all_intrp=False,show_final_intrp=False,show_sum=False)

Correction = 480/391.*1.1  #counted the blank for less time
bl_up,bl_dn = [i*Correction for i in (bl_up,bl_dn)]
Shim_blank,STD_blank = get_shim(bl_up,bl_dn,plot_parms=SHIM0_PLOT_PARMS)
Pol_blank,_ = get_pol(bl_up,bl_dn)

P0_PLOT_PARMS = {'title':'$P_0$ Pol','xlab':r'$\lambda$','ylab':r'Pixel','size':(12,8),'vals':(.5,1),\
                 'extent':(LAMBDAS[0],LAMBDAS[-1],PIXELS[0],PIXELS[-1]),'norm_max':False,'aspect':.25,'shrink':.25}
plot_2D(Pol_blank,plot_parms=P0_PLOT_PARMS)

fitted_q,_ = do_fit(Xt,Yt,Et,rebin=bl_lam)  #uses interpolated binning for compatibility with constant-q binning

Using the global parameters and fits from the above cells, we can now analyze the GH shift sample.

***

### Sample perpendicular, no grating (To do)
***

The shim intensity shows 2 distinct peaks. Technically, the beam has the same center, but I'm missing quite a few counts from pixel 8, so the P0 normalization needs to additionally include at least that pixel.

In [ ]:
_,PIXELS_PERP = get_center(Yup,Ydn,pname,rng=9)  #PIXELS_PERP doesn't include final pixel

X_perp,Y_perp,E_perp,pname = load_data_2D(r"blank\\pol.txt")

P0_PLOT_PARMS = {'title':'Pol','xlab':r'$\lambda$','ylab':r'Pixel','size':(12,8),'vals':(0,1),\
                  'extent':None,'norm_max':False,'aspect':None,'shrink':.25}

plot_2D(Y_perp,plot_parms=P0_PLOT_PARMS)

Xt_perp,Yt_perp,Et_perp = trim_arr((X_perp,Y_perp,E_perp),[(3.,12.)])  #trimming
Yt_perp = Yt_perp[PIXELS_PERP[0]:PIXELS_PERP[-1],:]
Et_perp = Et_perp[PIXELS_PERP[0]:PIXELS_PERP[-1],:]
Xt_perp = Xt_perp[PIXELS_PERP[0]:PIXELS_PERP[-1],:]
xt_perp,yt_perp = Xt_perp[0,:],np.arange(PIXELS_PERP[-1]-PIXELS_PERP[0])

plot_2D(Yt_perp,plot_parms=P0_PLOT_PARMS|{'aspect':.25,'extent':(xt_perp[0],xt_perp[-1],yt_perp[0],yt_perp[-1]),'vals':(.5,1)})

fitted_perp,fitted_extended_perp = do_fit(Xt_perp,Yt_perp,Et_perp)

In [ ]:
X_perp_p35,Yup_perp_p35,_,pname = load_data_2D(r"perp_p35\\up.txt")
_,Ydn_perp_p35,_,_ = load_data_2D(r"perp_p35\\dn.txt")

_ = get_center(Yup_perp_p35,Ydn_perp_p35,pname,rng=9)

Xt,Yup_perp_p35 = trim_arr((X,Yup_perp_p35),[(3.,12.)])
Yup_perp_p35 = Yup_perp_p35[PIXELS_PERP[0]:PIXELS_PERP[-1],:]

_,Ydn_perp_p35 = trim_arr((X,Ydn_perp_p35),[(3.,12.)])
Ydn_perp_p35 = Ydn_perp_p35[PIXELS_PERP[0]:PIXELS_PERP[-1],:]

plot_2D(shim_perp:=(Yup_perp_p35+Ydn_perp_p35),plot_parms=POL_PLOT_PARMS|{'title':'perp, no grating, shim','vals':(None,None)})
STD_perp = np.array([np.sqrt(np.cov(range(9),aweights=lam)) for lam in shim_perp.T])

Y_perp_p35,_ = get_pol(Yup_perp_p35,Ydn_perp_p35)
Y_perp_p35_n = Y_perp_p35/fitted_perp

plot_2D(Y_perp_p35_n,plot_parms=POL_PLOT_PARMS|{'title':'perp, no grating, normed'})

Yup_perp_p35_summed = np.sum(Yup_perp_p35,axis=0)
Ydn_perp_p35_summed = np.sum(Ydn_perp_p35,axis=0)
Y_perp_p35_summed,_ = get_pol(Yup_perp_p35_summed,Ydn_perp_p35_summed)
Y_perp_p35_summed_n = Y_perp_p35_summed/np.mean(fitted_perp,axis=0)

x_av,y_av,_ = load_data(r"perp_p35\\pol_av.txt")  #comparing my reduction to Rob's code
x_av,y_av = trim_arr((x_av,y_av),[(3.,12.)])
rob = y_av/np.mean(fitted_perp,axis=0)

plt.figure(figsize=(11,5))
#for indx,p in enumerate(Y_perp_p35_n):
    #plt.plot(xt_perp,p,label=f'{indx}')
#plt.plot(xt_perp,np.mean(Y_perp_p35_n,axis=0),label='Av',color='black')
plt.plot(xt_perp,Y_perp_p35_summed_n,label='Sum',color='red')
plt.plot(xt_perp,rob,'--',label='Rob',color='blue')
ax = plt.gca()
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.grid()
plt.ylim((-1.1,1.1))
plt.show()

In [ ]:
shim_perp_summed = np.sum(shim_perp,axis=0)

plt.figure(figsize=(10,4))
plt.title('No grating shim perp')
plt.xlabel('Wavelength')
for pix,s in enumerate(shim_perp):
    plt.plot(xt_perp,s,label=f'Pix {pix}')
#plt.plot(xt_perp,shim_perp_summed,label='sum',color='black')
plt.grid()
ax = plt.gca()
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

plt.figure(figsize=(10,4))
plt.title('No grating shim perp standard deviation')
plt.xlabel('Wavelength')
plt.plot(xt_perp,STD_perp,'.-')
plt.grid()
plt.show()

#perp_sigs,perp_errs,tests = do_sig_fit(shim_perp,tests=[170,179])  #fix harded coded values

'''plt.figure(figsize=(10,4))
plt.title('No grating perp standard deviations')
plt.plot(xt_perp,perp_sigs,'.',color='C0',label='fitted')
plt.plot(xt_perp,STD_perp,'.-',color='C1',label='calc')
plt.errorbar(xt_perp,perp_sigs,yerr=perp_errs,color='C0',ls='')
plt.axvline(x=xt_perp[tests[0]],ls='--',color='black')
plt.axvline(x=xt_perp[tests[-1]],ls='--',color='black')
ax = plt.gca()
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.xlabel('Wavelength')
plt.grid()
plt.show()'''

In [ ]:
for indx,(y,yn,f) in enumerate(zip(Y_perp_p35,Y_perp_p35_n,fitted_perp)):
    plt.figure(figsize=(11,4))
    plt.title('perp, no grating')
    #plt.plot(xt,y)
    plt.plot(xt_perp,yn,label=f'Pixel {indx} normed')
    plt.plot(xt_perp,Y_perp_p35_summed_n,'--',label='det average')
    plt.plot(xt_perp,f)
    plt.grid()
    plt.ylim((-1.1,1.1))
    plt.legend()
    plt.show()

In [ ]:
Q = Q_conversion(xt)[2:-1]
pol_Q = np.flip(Y_perp_p35_n,axis=1)[2:-1]

plt.figure(figsize=(11,4))
for q,p in zip(Q,pol_Q):
    plt.plot(q,p,'.-')
plt.grid()
plt.ylim((-1.1,1.1))
plt.show()

cs = sc.interpolate.CubicSpline
q_spaces = [np.linspace(q[0],q[-1],400) for q in Q]

for indx,(q,p) in enumerate(zip(Q,pol_Q)):
    plt.figure(figsize=(11,4))
    plt.plot(q,p,'o',label=f'Pixel {indx}')
    plt.plot(q_spaces[indx],cs(q,p)(q_spaces[indx]),'-',color='red')
    plt.grid()
    plt.ylim((-1.1,1.1))
    plt.legend()
    plt.close()

pol_intrp = [cs(q,p)(qs) for q,p,qs in zip(Q,pol_Q,q_spaces)]
plt.figure(figsize=(11,4))
for indx,p in enumerate(pol_intrp):
    plt.plot(q_spaces[indx],p)
plt.ylim((-1.1,1.1))
plt.grid()
plt.show()

In [ ]:
q_tot = np.linspace(np.min(Q),np.max(Q),400)
new_bins = np.digitize(q_spaces,q_tot,right=True)
temp = [do_rebin(p,nb) for p,nb in zip(pol_intrp,new_bins)]

plt.figure(figsize=(11,4))
plt.title('Pol normed, pixel averaged, constant q')
for t in temp:
    plt.plot(q_tot,t)
plt.plot(q_tot,no_grating_perp:=np.nanmean(temp,axis=0),color='black',label='Av')
plt.legend()
plt.grid()
plt.ylim((-1.1,1.1))
plt.show()

### Sample parallel, no grating
***

##### First method:

This method doesn't give the right results because the shim intensity varies too much over the detector, so I had to get the direct intensities, then bin them, and finally manually calculate the polarization.

In [ ]:
LAMS_para_p35,POL_para_p35,E_para_p35,pname_para_p35 = load_data_2D(r"para_p35\\pol.txt")

POL_PLOT_PARMS = DEFAULT_POL_PLOT_PARMS|{'title':'Para, no grating','extent':(LAMBDAS[0],LAMBDAS[-1],PIXELS[0],PIXELS[-1])}

LAMS_para_p35,POL_para_p35,E_para_p35 = trim_arr((LAMS_para_p35,POL_para_p35,E_para_p35),[(3.,12.)])
POL_para_p35 = POL_para_p35[EDGES[0]:EDGES[-1],:]  #choosing detector pixels, doesn't include final pixel
E_para_p35 = E_para_p35[EDGES[0]:EDGES[-1],:]
LAMS_para_p35 = LAMS_para_p35[EDGES[0]:EDGES[-1],:]

plot_2D(POL_para_p35,plot_parms=POL_PLOT_PARMS)

POL_para_p35_n = POL_para_p35/fitted
plot_2D(POL_para_p35_n,plot_parms=POL_PLOT_PARMS|{'title':'Para, no grating, normed'})

plt.figure(figsize=(11,5))
plt.title('Post pol binning (Bad!)')
for indx,p in enumerate(POL_para_p35_n):
    plt.plot(LAMBDAS,p,label=f'{indx}')
plt.plot(LAMBDAS,np.mean(POL_para_p35_n,axis=0),label='Av',color='black')
ax = plt.gca()
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.grid()
plt.ylim((-1.1,1.1))
plt.show()

##### Second method: 

Here I'm using "lam_1" and "lam_2" arrays from Mantid.

I still can't get the correct errors until I can understand the normalization (only by monitor counts?). Anyways, I'll use this method for the rest of the data analysis.

In [ ]:
LAMS_para_p35,Up_para_p35,Eup_para_p35,pname = load_data_2D(r"para_p35\\up.txt")
_,Dn_para_p35,Edn_para_p35,_ = load_data_2D(r"para_p35\\dn.txt")

_ = get_center(Up_para_p35,Dn_para_p35,pname)  #checking alignment

POL_PLOT_PARMS = DEFAULT_POL_PLOT_PARMS|{'title':'Para, no grating','extent':(LAMBDAS[0],LAMBDAS[-1],PIXELS[0],PIXELS[-1])}
SHIM_PLOT_PARMS = POL_PLOT_PARMS|{'title':'Para, no grating, shim','vals':(None,None),'log vals':(-8,-4)}

LAMS_para_p35,Up_para_p35,Eup_para_p35,Dn_para_p35,Edn_para_p35 = trim_arr((LAMS_para_p35,Up_para_p35,Eup_para_p35,\
                                                                           Dn_para_p35,Edn_para_p35),[(3.,12.)])
Up_para_p35 = Up_para_p35[EDGES[0]:EDGES[-1],:]  #uses global EDGES tuple
Eup_para_p35 = Eup_para_p35[EDGES[0]:EDGES[-1],:]
Dn_para_p35 = Dn_para_p35[EDGES[0]:EDGES[-1],:]
Edn_para_p35 = Edn_para_p35[EDGES[0]:EDGES[-1],:]

plot_2D(Up_para_p35,plot_parms=SHIM_PLOT_PARMS|{'title':'Raw up'})
plot_2D(Dn_para_p35,plot_parms=SHIM_PLOT_PARMS|{'title':'Raw down'})
plot_2D(Up_para_p35+Dn_para_p35,plot_parms=SHIM_PLOT_PARMS|{'title':'Raw shim'})

para_p35_lam,Up_para_p35,Up_para_p35_err = get_constant_Q(Up_para_p35,Eup_para_p35,LAMBDAS,th0=.35,trim=(1,-1),lims=(None,None),\
                                                          show_all=False,show_all_intrp=False,show_final_intrp=False,show_sum=True)
_,Dn_para_p35,Dn_para_p35_err = get_constant_Q(Dn_para_p35,Edn_para_p35,LAMBDAS,th0=.35,trim=(1,-1),lims=(None,None),\
                                               show_all=False,show_all_intrp=False,show_final_intrp=False,show_sum=True)

Shim_para_p35,STD_para_p35 = get_shim(Up_para_p35,Dn_para_p35,plot_parms=SHIM_PLOT_PARMS)

Pol_para_p35,_ = get_pol(Up_para_p35,Dn_para_p35)
Pol_para_p35 = Pol_para_p35/fitted_q[1:-1]  #normalize by P0

plot_2D(Pol_para_p35,plot_parms=POL_PLOT_PARMS|{'title':'Para, no grating, pol'})

In [ ]:
Para_p35_shim_s = np.sum(Shim_para_p35[1:,:],axis=0)/np.sum(Shim_blank[1:,:],axis=0)
Para_p35_pol_m = np.mean(Pol_para_p35[1:,:],axis=0)

plt.figure(figsize=(11,4))
plt.title('Pol (-) and shim (--), no grating')
if False:
    for indx,(p,s,sb) in enumerate(zip(Pol_para_p35[1:,:],Shim_para_p35[1:,:],Shim_blank[1:,:])):
        plt.plot(para_p35_lam,p,label=f'{indx}',color=f'C{indx}')
        plt.plot(para_p35_lam,s/sb,'--',color=f'C{indx}')
plt.plot(para_p35_lam,Para_p35_pol_m,color='black',label='av')
plt.plot(para_p35_lam,Para_p35_shim_s,'--',color='black')
plt.grid()
ax = plt.gca()
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.ylim((-1.1,1.1))
plt.show()

##### Comparing my reduction method to Rob's code in the Mantid reduction:

It looks like the difference between the two methods was that rob was summing up the entire detector instead of the pixels of interest.

In [ ]:
def compare_1d(Yup,Ydn,rob,lam,fitted=None,show=True):
    '''Comparing my reduction method to Rob's code in the Mantid reduction 
    for the 1d pixel-averaged polarization.'''
    Y_summed,_ = get_pol(np.sum(Yup,axis=0),np.sum(Ydn,axis=0))
    if fitted is not None:
        Y_summed = Y_summed/np.mean(fitted,axis=0)  #optional normalization
    if show:
        plt.figure(figsize=(10,4))
        plt.title('Average vs summed methods')
        plt.plot(lams[0,:],Y_summed,label='Sum',color='red')
        plt.plot(lams[0,:],rob,'--',label='Av',color='blue')
        plt.grid()
        plt.ylim((-1.1,1.1))
        plt.legend()
        plt.show()
    return Y_summed

x_av,y_av,_ = load_data(r"para_p35\\pol_av.txt")  #Rob's method here
x_av,y_av = trim_arr((x_av,y_av),[(3.,12.)])
rob = y_av/np.mean(fitted,axis=0)

lams,up,_,_ = load_data_2D(r"para_p35\\up.txt")
_,dn,_,_ = load_data_2D(r"para_p35\\dn.txt")
lams,up,dn = trim_arr((lams,up,dn),[(3.,12.)])
up = up[EDGES[0]:EDGES[-1],:]
dn = dn[EDGES[0]:EDGES[-1],:]

Para_p35_summed_n = compare_1d(up,dn,rob,lams,fitted)

##### Looking at the standard deviations:

The second method (fitting to a Gaussian) to find the beam width as a function of wavelength works better for this data, but is extremely poor for the data with the gratings.

In [ ]:
plt.figure(figsize=(10,4)) #para_p35_lam
plt.title('No grating shim')
for pix,s in enumerate(Shim_para_p35[1:,:]/Shim_blank[1:,:]):
    plt.plot(para_p35_lam,s,label=f'Pix {pix}')
plt.plot(para_p35_lam,Para_p35_shim_s,label='sum',color='black')
plt.ylim((0,1.1))
plt.xlabel('Wavelength')
plt.grid()
ax = plt.gca()
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

plt.figure(figsize=(10,4))
plt.title('No grating shim standard deviation')
plt.xlabel('Wavelength')
plt.plot(para_p35_lam,STD_para_p35,'.-')
plt.grid()
plt.show()

para_sigs,para_errs,tests = do_sig_fit(Shim_para_p35,para_p35_lam,tests=[250,260])

plt.figure(figsize=(10,4))
plt.title('No grating standard deviations')
plt.plot(para_p35_lam,para_sigs,'.',color='C0',label='fitted')
plt.plot(para_p35_lam,STD_para_p35,'.-',color='C1',label='calc')
plt.errorbar(para_p35_lam,para_sigs,yerr=para_errs,color='C0',ls='')
plt.axvline(x=para_p35_lam[tests[0]],ls='--',color='black')
plt.axvline(x=para_p35_lam[tests[-1]],ls='--',color='black')
ax = plt.gca()
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.xlabel('Wavelength')
plt.ylim((.5,1.5))
plt.grid()
plt.show()

### Sample parallel, grating before
***

In [ ]:
LAMS_para_p35_before,Up_para_p35_before,Eup_para_p35_before,pname = load_data_2D(r"para_p35_before\\up.txt")
_,Dn_para_p35_before,Edn_para_p35_before,_ = load_data_2D(r"para_p35_before\\dn.txt")

_ = get_center(Up_para_p35_before,Dn_para_p35_before,pname)  #checking alignment

POL_PLOT_PARMS = DEFAULT_POL_PLOT_PARMS|{'title':'Para, grating before','extent':(LAMBDAS[0],LAMBDAS[-1],PIXELS[0],PIXELS[-1])}
SHIM_PLOT_PARMS = POL_PLOT_PARMS|{'title':'Para, grating before, shim','vals':(None,None),'log vals':(-8,-4)}

LAMS_para_p35_before,Up_para_p35_before,Eup_para_p35_before,Dn_para_p35_before,Edn_para_p35_before = trim_arr((LAMS_para_p35_before,Up_para_p35_before,Eup_para_p35_before,Dn_para_p35_before,Edn_para_p35_before),[(3.,12.)])

Up_para_p35_before = Up_para_p35_before[EDGES[0]:EDGES[-1],:]  #uses global EDGES tuple
Eup_para_p35_before = Eup_para_p35_before[EDGES[0]:EDGES[-1],:]
Dn_para_p35_before = Dn_para_p35_before[EDGES[0]:EDGES[-1],:]
Edn_para_p35_before = Edn_para_p35_before[EDGES[0]:EDGES[-1],:]

plot_2D(Up_para_p35_before,plot_parms=SHIM_PLOT_PARMS|{'title':'Raw up'})
plot_2D(Dn_para_p35_before,plot_parms=SHIM_PLOT_PARMS|{'title':'Raw down'})
plot_2D(Up_para_p35_before+Dn_para_p35_before,plot_parms=SHIM_PLOT_PARMS|{'title':'Raw shim'})

plot_2D(no_q_pol_before:=get_pol(Up_para_p35_before,Dn_para_p35_before)[0],plot_parms=POL_PLOT_PARMS|{'title':'Para, grating before, pol pre-q binning'})

para_p35_before_lam,Up_para_p35_before,Up_para_p35_err_before = get_constant_Q(Up_para_p35_before,Eup_para_p35_before,LAMBDAS,th0=.35,trim=(1,-1),lims=(None,None),\
                                                                               show_all=False,show_all_intrp=False,show_final_intrp=False,show_sum=True)
_,Dn_para_p35_before,Dn_para_p35_err_before = get_constant_Q(Dn_para_p35_before,Edn_para_p35_before,LAMBDAS,th0=.35,trim=(1,-1),lims=(None,None),\
                                                             show_all=False,show_all_intrp=False,show_final_intrp=False,show_sum=True)

Shim_para_p35_before,STD_para_p35_before = get_shim(Up_para_p35_before,Dn_para_p35_before,plot_parms=SHIM_PLOT_PARMS)

Pol_para_p35_before,_ = get_pol(Up_para_p35_before,Dn_para_p35_before)
Pol_para_p35_before = Pol_para_p35_before/fitted_q[1:-1]  #normalize by P0

plot_2D(Pol_para_p35_before,plot_parms=POL_PLOT_PARMS|{'title':'Para, grating before, pol'})

In [ ]:
Para_p35_before_shim_s = np.sum(Shim_para_p35_before[1:,:],axis=0)/np.sum(Shim_blank[1:,:],axis=0)
Para_p35_before_pol_m = np.mean(Pol_para_p35_before[1:,:],axis=0)

plt.figure(figsize=(11,4))
plt.title('Pol (-) and shim (--), grating before')
if False:
    for indx,(p,s,sb) in enumerate(zip(Pol_para_p35_before[1:,:],Shim_para_p35_before[1:,:],Shim_blank[1:,:])):
        plt.plot(para_p35_before_lam,p,label=f'{indx}',color=f'C{indx}')
        plt.plot(para_p35_before_lam,s/sb,'--',color=f'C{indx}')
plt.plot(para_p35_lam,Para_p35_before_pol_m,color='black',label='av')
plt.plot(para_p35_lam,Para_p35_before_shim_s,'--',color='black')
plt.grid()
ax = plt.gca()
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.ylim((-1.1,1.1))
plt.show()

##### Rob's code comparison:

Notice that the blue (data directly from the Mantid reduction script) and yellow (manually averaged over the entire detector from the 2D data) lines are equivalent. Therefore, the Mantid reduction is summing over the entire detector, and not just the middle 7 pixels as we previously thought.

This also shows that we are getting a noticeable scattering contribution from the grating, even before the sample.

In [ ]:
x_av,y_av,_ = load_data(r"para_p35_before\\pol_av.txt")  #Rob's method here
x_av,y_av = trim_arr((x_av,y_av),[(3.,12.)])
rob = y_av/np.mean(fitted,axis=0)

lams,up,_,_ = load_data_2D(r"para_p35_before\\up.txt")
_,dn,_,_ = load_data_2D(r"para_p35_before\\dn.txt")
lams,up,dn = trim_arr((lams,up,dn),[(3.,12.)])
up = up[EDGES[0]:EDGES[-1],:]
dn = dn[EDGES[0]:EDGES[-1],:]

Para_p35_before_summed_n = compare_1d(up,dn,rob,lams,fitted)

##### Standard deviation check:

Seems like there is some spreading, so I need to check the expanded shim.

In [ ]:
plt.figure(figsize=(10,4)) #para_p35_lam
plt.title('Grating before shim')
for pix,s in enumerate(Shim_para_p35_before[1:,:]/Shim_blank[1:,:]):
    plt.plot(para_p35_lam,s,label=f'Pix {pix}')
plt.plot(para_p35_lam,Para_p35_before_shim_s,label='sum',color='black')
plt.ylim((0,1.1))
plt.xlabel('Wavelength')
plt.grid()
ax = plt.gca()
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

plt.figure(figsize=(10,4))
plt.title('Grating before shim standard deviation')
plt.xlabel('Wavelength')
plt.plot(para_p35_before_lam,STD_para_p35_before,'.-')
plt.grid()
plt.show()

para_sigs,para_errs,tests = do_sig_fit(Shim_para_p35_before,para_p35_before_lam,tests=[250,260],use_shim=True)

plt.figure(figsize=(10,4))
plt.title('Grating before standard deviations')
plt.plot(para_p35_before_lam,para_sigs,'.',color='C0',label='fitted')
plt.plot(para_p35_before_lam,STD_para_p35_before,'.-',color='C1',label='calc')
plt.errorbar(para_p35_before_lam,para_sigs,yerr=para_errs,color='C0',ls='')
plt.axvline(x=para_p35_before_lam[tests[0]],ls='--',color='black')
plt.axvline(x=para_p35_before_lam[tests[-1]],ls='--',color='black')
ax = plt.gca()
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.xlabel('Wavelength')
plt.ylim((0.,2.))
plt.grid()
plt.show()

In [ ]:
_,up_ng,_,pname = load_data_2D(r"para_p35\\up.txt")
_,dn_ng,_,_ = load_data_2D(r"para_p35\\dn.txt")

lam,up,_,pname = load_data_2D(r"para_p35_before\\up.txt")
_,dn,_,_ = load_data_2D(r"para_p35_before\\dn.txt")

temp_cut = [25,45]

lam,up,dn,up_ng,dn_ng = trim_arr((lam,up,dn,up_ng,dn_ng),[(3.,12.)])
lam,up,dn,up_ng,dn_ng = [k[temp_cut[0]:temp_cut[-1],:] for k in [lam,up,dn,up_ng,dn_ng]]

SHIM_PLOT_PARMS = POL_PLOT_PARMS|{'title':'Para, no grating, shim','vals':(None,None),'log vals':(-10,-4),\
                                 'extent':(LAMBDAS[0],LAMBDAS[-1],temp_cut[0],temp_cut[-1]),'aspect':.1}
_ = get_shim(up_ng,dn_ng,plot_parms=SHIM_PLOT_PARMS)

SHIM_PLOT_PARMS = SHIM_PLOT_PARMS|{'title':'Para, grating before, shim'}
_ = get_shim(up,dn,plot_parms=SHIM_PLOT_PARMS)

### Sample parallel, grating after

In [ ]:
LAMS_para_p35_after,Up_para_p35_after,Eup_para_p35_after,pname = load_data_2D(r"para_p35_after\\up.txt")
_,Dn_para_p35_after,Edn_para_p35_after,_ = load_data_2D(r"para_p35_after\\dn.txt")

_ = get_center(Up_para_p35_after,Dn_para_p35_after,pname)  #checking alignment

POL_PLOT_PARMS = DEFAULT_POL_PLOT_PARMS|{'title':'Para, grating after','extent':(LAMBDAS[0],LAMBDAS[-1],PIXELS[0],PIXELS[-1])}
SHIM_PLOT_PARMS = POL_PLOT_PARMS|{'title':'Para, grating after, shim','vals':(None,None),'log vals':(-8,-4)}

LAMS_para_p35_after,Up_para_p35_after,Eup_para_p35_after,Dn_para_p35_after,Edn_para_p35_after = trim_arr((LAMS_para_p35_after,Up_para_p35_after,Eup_para_p35_after,Dn_para_p35_after,Edn_para_p35_after),[(3.,12.)])

Up_para_p35_after = Up_para_p35_after[EDGES[0]:EDGES[-1],:]  #uses global EDGES tuple
Eup_para_p35_after = Eup_para_p35_after[EDGES[0]:EDGES[-1],:]
Dn_para_p35_after = Dn_para_p35_after[EDGES[0]:EDGES[-1],:]
Edn_para_p35_after = Edn_para_p35_after[EDGES[0]:EDGES[-1],:]

plot_2D(Up_para_p35_after,plot_parms=SHIM_PLOT_PARMS|{'title':'Raw up'})
plot_2D(Dn_para_p35_after,plot_parms=SHIM_PLOT_PARMS|{'title':'Raw down'})
plot_2D(Up_para_p35_after+Dn_para_p35_after,plot_parms=SHIM_PLOT_PARMS|{'title':'Raw shim'})

plot_2D(no_q_pol_after:=get_pol(Up_para_p35_after,Dn_para_p35_after)[0],plot_parms=POL_PLOT_PARMS|{'title':'Para, grating after, pol pre-q binning'})

para_p35_after_lam,Up_para_p35_after,Up_para_p35_err_after = get_constant_Q(Up_para_p35_after,Eup_para_p35_after,LAMBDAS,th0=.35,trim=(1,-1),lims=(None,None),\
                                                                               show_all=False,show_all_intrp=False,show_final_intrp=False,show_sum=True)
_,Dn_para_p35_after,Dn_para_p35_err_after = get_constant_Q(Dn_para_p35_after,Edn_para_p35_after,LAMBDAS,th0=.35,trim=(1,-1),lims=(None,None),\
                                                             show_all=False,show_all_intrp=False,show_final_intrp=False,show_sum=True)

Shim_para_p35_after,STD_para_p35_after = get_shim(Up_para_p35_after,Dn_para_p35_after,plot_parms=SHIM_PLOT_PARMS)

Pol_para_p35_after,_ = get_pol(Up_para_p35_after,Dn_para_p35_after)
Pol_para_p35_after = Pol_para_p35_after/fitted_q[1:-1]  #normalize by P0

plot_2D(Pol_para_p35_after,plot_parms=POL_PLOT_PARMS|{'title':'Para, grating after, pol'})

In [ ]:
Para_p35_after_shim_s = np.sum(Shim_para_p35_after[1:,:],axis=0)/np.sum(Shim_blank[1:,:],axis=0)
Para_p35_after_pol_m = np.mean(Pol_para_p35_after[1:,:],axis=0)

plt.figure(figsize=(11,4))
plt.title('Pol (-) and shim (--), grating after')
if True:
    for indx,(p,s,sb) in enumerate(zip(Pol_para_p35_after[1:,:],Shim_para_p35_after[1:,:],Shim_blank[1:,:])):
        plt.plot(para_p35_after_lam,p,label=f'{indx}',color=f'C{indx}')
        plt.plot(para_p35_after_lam,s/sb,'--',color=f'C{indx}')
plt.plot(para_p35_lam,Para_p35_after_pol_m,color='black',label='av')
plt.plot(para_p35_lam,Para_p35_after_shim_s,'--',color='black')
plt.grid()
ax = plt.gca()
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.ylim((-1.1,1.1))
plt.show()

In [ ]:
plt.figure(figsize=(10,4)) #para_p35_lam
plt.title('Grating after shim')
for pix,s in enumerate(Shim_para_p35_after[1:,:]/Shim_blank[1:,:]):
    plt.plot(para_p35_lam,s,label=f'Pix {pix}')
plt.plot(para_p35_lam,Para_p35_after_shim_s,label='sum',color='black')
plt.ylim((0,1.1))
plt.xlabel('Wavelength')
plt.grid()
ax = plt.gca()
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

plt.figure(figsize=(10,4))
plt.title('Grating after shim standard deviation')
plt.xlabel('Wavelength')
plt.plot(para_p35_after_lam,STD_para_p35_after,'.-')
plt.grid()
plt.show()

para_sigs,para_errs,tests = do_sig_fit(Shim_para_p35_after,para_p35_after_lam,tests=[250,260],use_shim=True)

plt.figure(figsize=(10,4))
plt.title('Grating after standard deviations')
plt.plot(para_p35_after_lam,para_sigs,'.',color='C0',label='fitted')
plt.plot(para_p35_after_lam,STD_para_p35_after,'.-',color='C1',label='calc')
plt.errorbar(para_p35_after_lam,para_sigs,yerr=para_errs,color='C0',ls='')
plt.axvline(x=para_p35_after_lam[tests[0]],ls='--',color='black')
plt.axvline(x=para_p35_after_lam[tests[-1]],ls='--',color='black')
ax = plt.gca()
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.xlabel('Wavelength')
plt.ylim((0.,2.))
plt.grid()
plt.show()

##### The spread due to Bragg scattering with the grating behind the GH sample is much more apparent, as expected.

In [ ]:
_,up_ng,_,pname = load_data_2D(r"para_p35\\up.txt")
_,dn_ng,_,_ = load_data_2D(r"para_p35\\dn.txt")

lam,up,_,_ = load_data_2D(r"para_p35_before\\up.txt")
_,dn,_,_ = load_data_2D(r"para_p35_before\\dn.txt")

lam,up_af,_,_ = load_data_2D(r"para_p35_after\\up.txt")
_,dn_af,_,_ = load_data_2D(r"para_p35_after\\dn.txt")

temp_cut = [25,45]

lam,up,dn,up_ng,dn_ng,up_af,dn_af = trim_arr((lam,up,dn,up_ng,dn_ng,up_af,dn_af),[(3.,12.)])
lam,up,dn,up_ng,dn_ng,up_af,dn_af = [k[temp_cut[0]:temp_cut[-1],:] for k in [lam,up,dn,up_ng,dn_ng,up_af,dn_af]]

SHIM_PLOT_PARMS = POL_PLOT_PARMS|{'title':'Para, no grating, shim','vals':(None,None),'log vals':(-10,-4),\
                                 'extent':(LAMBDAS[0],LAMBDAS[-1],temp_cut[0],temp_cut[-1]),'aspect':.1}
_ = get_shim(up_ng,dn_ng,plot_parms=SHIM_PLOT_PARMS)

SHIM_PLOT_PARMS = SHIM_PLOT_PARMS|{'title':'Para, grating before, shim'}
_ = get_shim(up,dn,plot_parms=SHIM_PLOT_PARMS)

SHIM_PLOT_PARMS = SHIM_PLOT_PARMS|{'title':'Para, grating after, shim'}
_ = get_shim(up_af,dn_af,plot_parms=SHIM_PLOT_PARMS)

***
### 0.4 degree data (To do)

***
### Comparisons between the data sets:

When looking at the constant-q binning, the peak disappearance that we saw at ISIS does not appear.

##### Entire detector shim comparison:

In [ ]:
_,up_ng,_,pname = load_data_2D(r"para_p35\\up.txt")
_,dn_ng,_,_ = load_data_2D(r"para_p35\\dn.txt")

lam,up,_,_ = load_data_2D(r"para_p35_before\\up.txt")
_,dn,_,_ = load_data_2D(r"para_p35_before\\dn.txt")

lam,up_af,_,_ = load_data_2D(r"para_p35_after\\up.txt")
_,dn_af,_,_ = load_data_2D(r"para_p35_after\\dn.txt")

temp_cut = [25,45]

lam,up,dn,up_ng,dn_ng,up_af,dn_af = trim_arr((lam,up,dn,up_ng,dn_ng,up_af,dn_af),[(3.,12.)])
lam,up,dn,up_ng,dn_ng,up_af,dn_af = [k[temp_cut[0]:temp_cut[-1],:] for k in [lam,up,dn,up_ng,dn_ng,up_af,dn_af]]

SHIM_PLOT_PARMS = POL_PLOT_PARMS|{'title':'Para, no grating, shim','vals':(None,None),'log vals':(-10,-4),\
                                 'extent':(LAMBDAS[0],LAMBDAS[-1],temp_cut[0],temp_cut[-1]),'aspect':.085}
_ = get_shim(up_ng,dn_ng,plot_parms=SHIM_PLOT_PARMS)

SHIM_PLOT_PARMS = SHIM_PLOT_PARMS|{'title':'Para, grating before, shim'}
_ = get_shim(up,dn,plot_parms=SHIM_PLOT_PARMS)

SHIM_PLOT_PARMS = SHIM_PLOT_PARMS|{'title':'Para, grating after, shim'}
_ = get_shim(up_af,dn_af,plot_parms=SHIM_PLOT_PARMS)

In [ ]:
plt.figure(figsize=(10,4))
plt.title('Q-corrected total intensities')
plt.plot(para_p35_after_lam,Para_p35_shim_s,label='No grating')  #clean up this x-axis name
plt.plot(para_p35_after_lam,Para_p35_before_shim_s,label='Grating before')
plt.plot(para_p35_after_lam,Para_p35_after_shim_s,label='Grating after')
plt.legend()
plt.grid()
plt.ylabel('Normalized shim')
plt.xlabel('Wavelength')
plt.ylim((0,1))
plt.show()

bs_trim = range(5,len(para_p35_after_lam)-15)
plt.figure(figsize=(10,4))
plt.title('Q-corrected standard deviations')
plt.plot(para_p35_after_lam[bs_trim],STD_para_p35[bs_trim],'.-',label='No grating')
plt.plot(para_p35_after_lam[bs_trim],STD_para_p35_before[bs_trim],'.-',label='Grating before')
plt.plot(para_p35_after_lam[bs_trim],STD_para_p35_after[bs_trim],'.-',label='Grating after')
plt.legend()
plt.grid()
plt.ylabel('Stnd dev')
plt.xlabel('Wavelength')
plt.show()

plt.figure(figsize=(10,4))
plt.title('Q-corrected total intensities, central pixel')
plt.plot(para_p35_after_lam,Shim_para_p35[2,:]/Shim_blank[2,:],label='No grating')  #clean up this x-axis name
plt.plot(para_p35_after_lam,Shim_para_p35_before[2,:]/Shim_blank[2,:],label='Grating before')
plt.plot(para_p35_after_lam,Shim_para_p35_after[2,:]/Shim_blank[2,:],label='Grating after')
plt.legend()
plt.grid()
plt.ylabel('Normalized shim')
plt.xlabel('Wavelength')
plt.ylim((0,1.1))
plt.show()

##### Polarization comparison:

In [ ]:
plot_2D(Pol_para_p35,plot_parms=POL_PLOT_PARMS|{'title':'Para, no grating, pol'})
plot_2D(Pol_para_p35_before,plot_parms=POL_PLOT_PARMS|{'title':'Para, grating before, pol'})
plot_2D(Pol_para_p35_after,plot_parms=POL_PLOT_PARMS|{'title':'Para, grating after, pol'})

plot_2D(no_q_pol_before,plot_parms=POL_PLOT_PARMS|{'title':'Para, grating before, pol, no q-binning'})
plot_2D(no_q_pol_after,plot_parms=POL_PLOT_PARMS|{'title':'Para, grating after, pol, no q-binning'})

In [ ]:
plt.figure(figsize=(10,4))
plt.title('Q-corrected average polarizations')
plt.plot(para_p35_after_lam,Para_p35_pol_m,label='No grating')  #clean up this x-axis name
plt.plot(para_p35_after_lam,Para_p35_before_pol_m,label='Grating before')
plt.plot(para_p35_after_lam,Para_p35_after_pol_m,label='Grating after')
plt.ylabel('Normalized pol')
plt.xlabel('Wavelength')
plt.legend()
plt.grid()
plt.show()

plt.figure(figsize=(10,4))
plt.title('Q-corrected polarizations, central pixel')
plt.plot(para_p35_after_lam,Pol_para_p35[2,:],label='No grating')  #clean up this x-axis name
plt.plot(para_p35_after_lam,Pol_para_p35_before[2,:],label='Grating before')
plt.plot(para_p35_after_lam,Pol_para_p35_after[2,:],label='Grating after')
plt.ylabel('Normalized pol')
plt.xlabel('Wavelength')
plt.legend()
plt.grid()
plt.show()

##### Other comparions:

***
### Other calculations: